## Setup

In [1]:
!pip install -q transformers
!pip install -q einops

In [2]:
GLOBAL_SEED = 42

import os
os.environ['PYTHONHASHSEED'] = str(GLOBAL_SEED)
import sys

import pandas as pd
import numpy as np
from numpy import random as np_rnd
import random as rnd
import gc
import datetime
import copy
import pickle
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings("ignore")

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
from torch.optim import AdamW
from transformers import get_polynomial_decay_schedule_with_warmup

from einops import rearrange
from einops.layers.torch import Rearrange

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    # python random
    rnd.seed(seed)
    # numpy random
    np_rnd.seed(seed)
    # tf random
    try:
        tf_rnd.set_seed(seed)
    except:
        pass
    # RAPIDS random
    try:
        cupy.random.seed(seed)
    except:
        pass
    # pytorch random
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
    except:
        pass

def pickleIO(obj, src, op="w"):
    if op=="w":
        with open(src, op + "b") as f:
            pickle.dump(obj, f)
    elif op=="r":
        with open(src, op + "b") as f:
            tmp = pickle.load(f)
        return tmp
    else:
        print("unknown operation")
        return obj
    
def findIdx(data_x, col_names):
    return [int(i) for i, j in enumerate(data_x) if j in col_names]

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

In [4]:
class CFG:
    local = False
    debug = False
    
    epochs = 20
    early_stopping_rounds = max(10, epochs // 5)
    batch_size = 128
    number_of_labels = 10
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    eta = 5e-4
    weight_decay = 1e-4

if CFG.debug:
    CFG.epochs = 5

## Loading CIFAR10 Dataset

In [5]:
# Create an instance for training. 
# When we run this code for the first time, the CIFAR10 train dataset will be downloaded locally. 
train_ds = datasets.CIFAR10(root="./data", train=True, download=True)
train_mean = [train_ds.data[:,:,:,i].mean() for i in range(train_ds.data.shape[-1])]
del train_ds; gc.collect()

Files already downloaded and verified


72

**INFO: Use MinMax Normalization 0~1 (different from the original paper)**

In [6]:
# Create Train dataset loader
train_ft = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32),
    transforms.Normalize(0, 255),
    # transforms.Normalize(train_mean, 1),
    transforms.RandomHorizontalFlip(),
])
train_dl = DataLoader(
    datasets.CIFAR10(root="./data", train=True, transform=train_ft ,download=True),
    batch_size=CFG.batch_size, shuffle=True, drop_last=True
)

# Create Test dataset loader
test_ft = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float32),
    transforms.Normalize(0, 255),
    # transforms.Normalize(train_mean, 1),
    # transforms.RandomHorizontalFlip(),
])
test_dl = DataLoader(
    datasets.CIFAR10(root="./data", train=False, transform=test_ft, download=True),
    batch_size=CFG.batch_size, shuffle=False, drop_last=False
)

print("The number of images in a test set is: ", len(test_dl) * CFG.batch_size)
print("The number of iteration is: ", len(train_dl))

Files already downloaded and verified
Files already downloaded and verified
The number of images in a test set is:  10112
The number of iteration is:  390


## Create Model & Define helper functions

In [7]:
def get_optimizer_params(model, eta, weight_decay):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        # apply weight decay
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': weight_decay},
        # don't apply weight decay for LayerNormalization layer
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'lr': eta, 'weight_decay': 0.0},
    ]
    return optimizer_parameters

def get_scheduler(optimizer, num_warmup_steps, num_training_steps):
    scheduler = get_polynomial_decay_schedule_with_warmup(
        optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps, power=0.5, lr_end=1e-7
    )
    return scheduler

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

**INFO: The model is simplified architecture from original paper**


In [8]:
img_size = 32
patch_size = 4
patch_dim = patch_size ** 2 * 3
seq_len = (img_size // 4) ** 2
num_heads = 2
base_hidden_layers = 64
assert img_size % patch_size == 0

class PatchExtract(nn.Module):
    def __init__(self):
        super(PatchExtract, self).__init__()
        self.get_patches = Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_size, p2=patch_size)

    def forward(self, x):
        x = self.get_patches(x)
        return x

class PatchEmbedding(nn.Module):
    def __init__(self, embed_dim):
        super(PatchEmbedding, self).__init__()

        self.pos_vector = torch.arange(seq_len)
        self.pos_linear = (self.pos_vector / seq_len).view(-1, 1)
        self.pos_center = (1 - ((seq_len / 2 - self.pos_vector).abs() / seq_len / 2)).view(-1, 1)
        self.pos_sin = torch.sin(self.pos_vector).view(-1, 1)
        self.pos_cos = torch.cos(self.pos_vector).view(-1, 1)
        self.pos_embed = nn.Embedding(seq_len, max(4, seq_len // 4))
        self.cls_token = nn.Parameter(torch.randn(1, embed_dim))
        self.lin_prj = nn.Linear(patch_dim + seq_len // 4 + 4, embed_dim)

    def forward(self, x):
        # Concatenate positional features
        # Linear, Center, Sin, Cos
        x_pos = torch.cat([
            self.pos_embed(self.pos_vector),
            self.pos_linear,
            self.pos_center,
            self.pos_sin,
            self.pos_cos,
        ], dim=-1)
        x_pos = x_pos.expand(x.shape[0], x_pos.shape[0], x_pos.shape[1])
        x = self.lin_prj(torch.cat([x, x_pos], dim=-1))
        x = torch.cat([self.cls_token.expand(x.shape[0], -1, -1), x], dim=1)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, dropoutRate_lin=0.25, dropoutRate_attn=0.125):
        super(TransformerBlock, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropoutRate_attn, batch_first=True)
        self.lin_prj = nn.Sequential(
            nn.Dropout(dropoutRate_lin),
            nn.Linear(embed_dim, embed_dim),
            nn.GELU()
        )
        self.layer_norm1 = nn.LayerNorm([seq_len + 1, embed_dim])
        self.layer_norm2 = nn.LayerNorm([seq_len + 1, embed_dim])
        
    def forward(self, x):
        x_norm = self.layer_norm1(x)
        x = self.attn(x_norm, x_norm, x_norm)[0] + x
        x_norm = self.layer_norm2(x)
        x = self.lin_prj(x_norm) + x
        return x

class ViT(nn.Module):
    def __init__(self):
        super(ViT, self).__init__()
        
        self.patch_extract = PatchExtract()
        self.patch_embedding = PatchEmbedding(base_hidden_layers)
        self.transformer_block = TransformerBlock(base_hidden_layers)

        self.fc = nn.Sequential(
            nn.Dropout(),
            nn.Linear(base_hidden_layers, base_hidden_layers // 4),
            nn.ReLU(),
        )

        self.classifier = nn.Linear(base_hidden_layers // 4, CFG.number_of_labels)

    def forward(self, x):
        x = self.patch_extract(x)
        x = self.patch_embedding(x)
        x = self.transformer_block(x)
        x = x[:, 0, :]
        x = self.fc(x)
        return self.classifier(x)

## Training

In [9]:
def fn_training():
    return_score_dic = []
    best_score = np.inf
    
    seed_everything()
    for epoch in range(CFG.epochs):
        train_loss = AverageMeter('Loss', ':.4e')
        valid_loss = AverageMeter('Loss', ':.4e')
        train_accuracy = AverageMeter('Accuracy', ':.4e')
        valid_accuracy = AverageMeter('Accuracy', ':.4e')
        train_f1 = AverageMeter('F1', ':.4e')
        valid_f1 = AverageMeter('F1', ':.4e')

        model.train()
        for feature, label in train_dl:
            with torch.cuda.amp.autocast():
                feature = feature.to(device)
                label = label.to(device)
                output = model(feature)
                loss = criterion(output, label)

            # initialization gradients to zero
            optimizer.zero_grad()
            # get scaled gradients by float16 (default)
            grad_scaler.scale(loss).backward()
            # apply original gradients (unscaling) to parameters
            # if these gradients do not contain infs or NaNs, optimizer.step() is then called.
            # otherwise, optimizer.step() is skipped.
            grad_scaler.step(optimizer)
            grad_scaler.update()
            scheduler.step()
            
            train_loss.update(loss.item())

            y_pred = output.argmax(axis=-1).detach().cpu().numpy()
            y_true = label.detach().cpu().numpy()

            train_accuracy.update(accuracy_score(y_true, y_pred))        
            train_f1.update(f1_score(y_true, y_pred, average="macro"))

        model.eval()
        for feature, label in test_dl:
            with torch.no_grad():
                feature = feature.to(device)
                label = label.to(device)
                output = model(feature)
                loss = criterion(output, label)

            valid_loss.update(loss.item())  

            y_pred = output.argmax(axis=-1).detach().cpu().numpy()
            y_true = label.detach().cpu().numpy()

            valid_accuracy.update(accuracy_score(y_true, y_pred))        
            valid_f1.update(f1_score(y_true, y_pred, average="macro"))

        score = valid_loss.avg
        return_score_dic.append({
            "epoch": epoch,
            "train_loss": train_loss.avg,
            "valid_loss": valid_loss.avg,
            "train_accuracy": train_accuracy.avg,
            "valid_accuracy": valid_accuracy.avg,
            "train_f1": train_f1.avg,
            "valid_f1": valid_f1.avg,  
        })

        if score < best_score:
            best_score = copy.deepcopy(score)
            early_stopping_cnt = 0 
        else:
            early_stopping_cnt += 1

        if early_stopping_cnt == CFG.early_stopping_rounds:
            print("INFO : Early Stopped ! (Epoch[{0}/{1}])".format(epoch+1, CFG.epochs))  
            break

        print(f'[{epoch+1:02d}/{CFG.epochs}]:  * Train Loss {train_loss.avg:.3f} * Train Accuracy {train_accuracy.avg:.3f} * Train F1 {train_f1.avg:.3f} * Valid Loss {valid_loss.avg:.3f} * Valid Accuracy {valid_accuracy.avg:.3f} * Valid F1 {valid_f1.avg:.3f}')

    return return_score_dic

In [10]:
# model
model = ViT()
model.to(device)

# optimizer & scheduler
optimizer_parameters = get_optimizer_params(
    model,
    eta=CFG.eta,
    weight_decay=CFG.weight_decay
)
optimizer = AdamW(optimizer_parameters, lr=CFG.eta, weight_decay=CFG.weight_decay)
scheduler = get_scheduler(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dl) * CFG.epochs)
)

# loss function
criterion = nn.CrossEntropyLoss()

# gradient scaler for fast operation with float16
grad_scaler = torch.cuda.amp.GradScaler()

In [11]:
%%time

score_dic = fn_training()

[01/20]:  * Train Loss 2.310 * Train Accuracy 0.101 * Train F1 0.073 * Valid Loss 2.304 * Valid Accuracy 0.101 * Valid F1 0.018
[02/20]:  * Train Loss 2.304 * Train Accuracy 0.102 * Train F1 0.076 * Valid Loss 2.302 * Valid Accuracy 0.100 * Valid F1 0.018
[03/20]:  * Train Loss 2.303 * Train Accuracy 0.102 * Train F1 0.078 * Valid Loss 2.300 * Valid Accuracy 0.100 * Valid F1 0.028
[04/20]:  * Train Loss 2.271 * Train Accuracy 0.123 * Train F1 0.085 * Valid Loss 2.188 * Valid Accuracy 0.158 * Valid F1 0.097
[05/20]:  * Train Loss 2.186 * Train Accuracy 0.160 * Train F1 0.120 * Valid Loss 2.145 * Valid Accuracy 0.174 * Valid F1 0.125
[06/20]:  * Train Loss 2.156 * Train Accuracy 0.172 * Train F1 0.135 * Valid Loss 2.128 * Valid Accuracy 0.189 * Valid F1 0.131
[07/20]:  * Train Loss 2.130 * Train Accuracy 0.191 * Train F1 0.153 * Valid Loss 2.080 * Valid Accuracy 0.235 * Valid F1 0.181
[08/20]:  * Train Loss 2.089 * Train Accuracy 0.226 * Train F1 0.195 * Valid Loss 2.025 * Valid Accuracy

In [12]:
pd.DataFrame(score_dic).mean().iloc[1:]

train_loss        2.058666
valid_loss        2.008859
train_accuracy    0.231303
valid_accuracy    0.255726
train_f1          0.200392
valid_f1          0.210210
dtype: float64